---
##**¡Búsqueda del tesoro!**
---
Autor: Juan Valverde Santiago

# Bienvenido a este puzzle.

En este juego, podrás programar el comportamiento de un agente para comprobar si es capaz de encontrar un tesoro en un laberinto.

Para realizar esta tarea, sigue los pasos que se indican en cada sección.

Pero primero, deberás guardarte una copia de este cuaderno, pues no puedes guardar cambios en el original.

# Descarga e importación

El siguiente bloque obtiene los archivos necesarios para la correcta ejecución del cuaderno.

Simplemente, ejecuta el bloque sin modificar nada.

In [1]:
%%capture
!mkdir v002
!mkdir images
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/v002/Agent.py' -O v002/Agent.py
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/v002/Enviroment.py' -O v002/Enviroment.py
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/v002/__init__.py' -O v002/__init__.py
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/v002/Enviroment_with_agents.py' -O v002/Enviroment_with_agents.py
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/v002/InOut_Simple_Laberinth.py' -O v002/InOut_Simple_Laberinth.py
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/images/PixelNoTomato.bmp' -O images/PixelNoTomato.bmp
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/images/PixelTomato.bmp' -O images/PixelTomato.bmp
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/images/face1_borders.bmp' -O images/face1_borders.bmp
!wget 'https://raw.githubusercontent.com/juanvs00/AgentSurvival/main/rules_to_clips_v3.py' -O rules_to_clips_v3.py

!pip install clipspy

from v002 import *
from rules_to_clips_v3 import *

# Escritura de las reglas

En este bloque debes trabajar. Aquí es donde debes escribir las reglas CLIPS y el nombre del agente.

Tu objetivo será crear un comportamiento para que los agentes peleen en un laberinto. No modifiques nombres de variables, céntrate en escribir las reglas que desees en la variable file.

Las percepciones del agente son los muros que le rodean y los objetos que encuentra, las opciones que puedes usar siguen la siguiente forma:
*   (walls \$? front \$?)
*   (not (walls \$? back \$?))
*   (available_object exit ?x)

Siendo los posibles muros front, back, left y right, que corresponden con los muros de delante, detrás, izquierda y derecha, respectivamente.

Y siendo los posibles objetos que encuentra el agente los siguientes:
*   exit
*   food type 1
*   agent

Por otro lado, las acciones que puede realizar el agente son las siguientes:
*   "self.move_forward()"
*   "self.turn_left()"
*   "self.turn_right()"
*   Funciones de objeto

Estas acciones deben añadirse en el hecho "pending_function_calls", como puedes ver en las reglas de ejemplo. En la causa de la regla se introduce el hecho en una variable; mientras que, en el consecuente se elimina el hecho y se vuelve a crear añadiendo las acciones que quieres que realice tu agente. En cuanto a las funciones de objeto, puedes observar la regla 4 de ejemplo llamada "salir" en la que se realiza la función de salida del laberinto.

Ten en cuenta que las acciones del agente consumen movimientos, y el número de movimientos está limitado.


Por último, también debes tener en cuenta el movimiento del agente modificando el hecho pos, que refiere a la posición del agente; y el hecho ori, que refiere hacia donde mira el agente. Lo que debes saber para resolver este problema es la estructura que usan estos datos:
*   (pos 0 0)
*   (ori 1 0)

Al comenzar el puzzle, la posición del agente(sea cuál sea) será la posición (0,0). En este juego la primera coordenada se refiere a la posición del eje Y y la segunda coordenada al eje X.

En cuanto a la orientación del agente pueden ser las siguientes:
*   Hacia arriba ---> (1, 0)
*   Hacia abajo ---> (-1, 0)
*   Hacia la izquierda ---> (0, -1)
*   Hacia la derecha ---> (0, 1)

Conociendo estos datos, debes introducir las operaciones para que el agente tenga sus hechos de posición y orientación correctos en todo momento. Esto es importante porque cuando el agente pasé por la misma posición 5 veces, se activará en el entorno el hecho (caution_loop), usado en la regla de ejemplo llamada "bucle".

**IMPORTANTE**

Debes usar el hecho (caution_loop) para programar el comportamiento del agente si es que quieres que tu agente encuentre el tesoro por lo general.


¡Suerte!

In [2]:
# TODO: Aquí debes escribir las reglas CLIPS
file = """
    (defrule giraDcha
    (not (walls $? right $?))
    ?h<-(pending_function_calls $?calls)
    ?h_pos<-(pos ?p1 ?p2)
    ?h_ori<-(ori ?v1 ?v2)
    (not (available_object exit ?x))
    (not (caution_loop))
    =>
    (retract ?h ?h_ori ?h_pos)
    (assert (pos (- ?p1 ?v2) (+ ?p2 ?v1)))
    (assert (ori (* -1 ?v2) ?v1))
    (assert (pending_function_calls $?calls "self.turn_right()" "self.move_forward()"))
    )

    (defrule haciaDelante
    (walls $? right $?)
    (not (walls $? front $?))
    ?h_pos<-(pos ?p1 ?p2)
    ?h_ori<-(ori ?v1 ?v2)
    ?h<-(pending_function_calls $?calls)
    (not (available_object exit ?x))
    (not (caution_loop))
    =>
    (retract ?h ?h_pos ?h_ori)
    (assert (pos (+ ?p1 ?v1) (+ ?p2 ?v2)))
    (assert (ori ?v1 ?v2))
    (assert (pending_function_calls $?calls "self.move_forward()"))
    )

    (defrule giraIzqda
    (walls $? right $?)
    (walls $? front $?)
    (not (walls $? left $?))
    ?h_pos<-(pos ?p1 ?p2)
    ?h_ori<-(ori ?v1 ?v2)
    ?h<-(pending_function_calls $?calls)
    (not (available_object exit ?x))
    (not (caution_loop))
    =>
    (retract ?h ?h_pos ?h_ori)
    (assert (pos (+ ?p1 ?v2) (- ?p2 ?v1)))
    (assert (ori ?v2 (* -1 ?v1)))
    (assert (pending_function_calls $?calls "self.turn_left()" "self.move_forward()"))
    )

    (defrule perdido
    (walls $? right $?)
    (walls $? front $?)
    (walls $? left $?)
    ?h_pos<-(pos ?p1 ?p2)
    ?h_ori<-(ori ?v1 ?v2)
    ?h<-(pending_function_calls $?calls)
    (not (available_object exit ?x))
    =>
    (retract ?h ?h_pos ?h_ori)
    (assert (pos (- ?p1 ?v1) (- ?p2 ?v2)))
    (assert (ori (* -1 ?v1) (* -1 ?v2)))
    (assert (pending_function_calls $?calls "self.turn_left()" "self.turn_left()" "self.move_forward()"))
    )

    (defrule salir
    (available_object exit ?x)
    ?h<-(pending_function_calls $?calls)
    =>
    (retract ?h)
    (assert (pending_function_calls ?x))
    )

    (defrule bucle
    (not (walls $? left $?))
    ?l<-(caution_loop)
    ?h_pos<-(pos ?p1 ?p2)
    ?h_ori<-(ori ?v1 ?v2)
    ?h<-(pending_function_calls $?calls)
    =>
    (retract ?h ?h_pos ?h_ori ?l)
    (assert (pos (+ ?p1 ?v2) (- ?p2 ?v1)))
    (assert (ori ?v2 (* -1 ?v1)))
    (assert (pending_function_calls $?calls "self.turn_left()" "self.move_forward()"))
    )

    (defrule bucle2
    (walls $? left $?)
    (not (walls $? front $?))
    ?l<-(caution_loop)
    ?h_pos<-(pos ?p1 ?p2)
    ?h_ori<-(ori ?v1 ?v2)
    ?h<-(pending_function_calls $?calls)
    =>
    (retract ?h ?h_pos ?h_ori ?l)
    (assert (pos (+ ?p1 ?v1) (+ ?p2 ?v2)))
    (assert (ori ?v1 ?v2))
    (assert (pending_function_calls $?calls "self.move_forward()"))
    )

    (defrule bucle3
    (walls $? left $?)
    (walls $? front $?)
    (not (walls $? right $?))
    ?l<-(caution_loop)
    ?h_pos<-(pos ?p1 ?p2)
    ?h_ori<-(ori ?v1 ?v2)
    ?h<-(pending_function_calls $?calls)
    =>
    (retract ?h ?h_pos ?h_ori ?l)
    (assert (pos (- ?p1 ?v2) (+ ?p2 ?v1)))
    (assert (ori (* -1 ?v2) ?v1))
    (assert (pending_function_calls $?calls "self.turn_right()" "self.move_forward()"))
    )"""

with open("rules.CLP", "w") as txt:
    print(format(file), file=txt)

rules= "rules.CLP"

# TODO: Aquí debes escribir el nombre de tu agente
agent_name = "YO!"

# Función move

El siguiente bloque corresponde con la función move del agente. Simplemente, ejecuta los bloques de código.

**¡NO DEBES MODIFICARLOS!**

In [3]:
runner = Agent_CLIPS(rules)

El siguiente bloque es la función move del agente, no modificar.

In [4]:
def move(self, available_objects=None):

  # Actualizar el entorno CLIPS y obtener las interacciones del agente
  walls, objects = runner.agent_context(self)

  # Obtener las funciones y los objetos gracias al algoritmo
  functions, available_objects = algorithm(objects, runner.env, available_objects)

  # Ejecutar las funciones en pending_function_calls
  for j in functions:
      j = j.replace('"','')
      eval(j)

# Ejecución del puzzle

Ha llegado el momento de ejecutar el puzzle para comprobar el comportamiento de tu agente.

En principio, no debes modificar nada. Aunque puedes modificar algún parámetro de la función InOut_Simple_Laberinth, por ejemplo el primer parámetro es el tamaño del escenario.

In [ ]:
reset_position(runner.env, runner.memory)
lb1 = InOut_Simple_Laberinth(10, plot_run='always',
                                 remove_walls_prob=0.5,
                                 exit_at_border=False,
                                 entry_at_border=False)

lb1.create_agent(agent_name, move)
lb1.run()

# ¿Es realmente bueno el comportamiento de tu agente?
En el siguiente bloque puedes comprobar cuántas veces es tu agente capaz de escapar del laberinto de un total de ejecuciones.

En principio, no debes modificar nada. Aunque puedes puedes modificar el número de ejecucuciones totales (la variable num_runs).

In [ ]:
from tqdm import tqdm
num_runs = 200
num_success =  0

for _ in tqdm(range(num_runs)):
  lb1 = InOut_Simple_Laberinth(10, plot_run='never',
                                 remove_walls_prob=0.5,
                                 exit_at_border=False,
                                 entry_at_border=False)
  lb1.create_agent(agent_name, move)
  winner = lb1.run()

  if winner is not None:
    num_success += 1

print("\n\nEncontraste la salida en", num_success, "ocasiones de", num_runs)